In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession 
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json
import time

In [2]:
%run ./TopicModeling.ipynb

In [3]:
ss = SparkSession.Builder() \
     .appName("Realtime") \
     .master("spark://spark-master-2:7077") \
     .config("spark.jars", "./spark-streaming-kafka-0-8-assembly_2.11-2.0.0-preview.jar") \
     .getOrCreate()
sc = ss.sparkContext
ssc = StreamingContext(sc, 10)
ssc.checkpoint("checkpoint_TwitterApp")
ss.sparkContext.setLogLevel('WARN')

In [4]:
brokers = 'kafka-1:9092,kafka-2:9092'
topic='tweet'

In [5]:
ks = KafkaUtils.createDirectStream(ssc, [topic], kafkaParams={"metadata.broker.list":brokers})

In [6]:
from pymongo import MongoClient
conn = MongoClient('mongodb', 27017)
mydb = conn["Tweet"]
mycol = mydb["realtime"]
conn.close()

In [7]:
def insert_row(x):
    if x is None or len(x)<1:
        return
    data_list = x.split(',')
    conn = MongoClient('mongodb', 27017)
    db = conn.Tweet
    db.realtime.insert_one({
        'time': data_list[0],
        'user': data_list[1],
        'place': data_list[2],
        'text': data_list[3],
        'topic': data_list[4],
    })
    conn.close()

In [8]:
lines = ks.map(
    lambda x: '{},{},{},{},{}'.format(
        json.loads(x[1])['time'],
        json.loads(x[1])['user'],
        json.loads(x[1])['place'],
        json.loads(x[1])['text'],
        classify(json.loads(x[1])['text'])
    )
)
lines.foreachRDD(lambda rdd: rdd.foreach(insert_row))
ssc.start()
ssc.awaitTermination()

Py4JJavaError: An error occurred while calling o42.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/local/spark/python/pyspark/streaming/dstream.py", line 161, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "<ipython-input-8-5de4660237aa>", line 10, in <lambda>
    lines.foreachRDD(lambda rdd: rdd.foreach(insert_row))
  File "/usr/local/spark/python/pyspark/rdd.py", line 789, in foreach
    self.mapPartitions(processPartition).count()  # Force evaluation
  File "/usr/local/spark/python/pyspark/rdd.py", line 1055, in count
    return self.mapPartitions(lambda i: [sum(1 for _ in i)]).sum()
  File "/usr/local/spark/python/pyspark/rdd.py", line 1046, in sum
    return self.mapPartitions(lambda x: [sum(x)]).fold(0, operator.add)
  File "/usr/local/spark/python/pyspark/rdd.py", line 917, in 